<a href="https://colab.research.google.com/github/hamuya/mnist/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow pillow matplotlib numpy

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# データセットをロード
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# データを前処理
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# モデルを構築
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# モデルをコンパイル
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# モデルを学習
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1)

# モデルを評価
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# モデルを保存
model.save('mnist_model.h5')


In [ ]:
import tensorflow as tf

# 学習済みモデルをロード
model = tf.keras.models.load_model('mnist_model.h5')


In [ ]:
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt

# 画像をアップロード
uploaded = files.upload()

# アップロードされた画像のファイル名を取得
image_path = list(uploaded.keys())[0]

In [ ]:
import numpy as np
from PIL import Image, ImageOps

# 手書き数字の画像を前処理
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    img = ImageOps.invert(img)
    img = img.resize((28, 28))
    img = np.array(img).astype('float32') / 255
    img = img.reshape(1, 28, 28, 1)
    return img

# 手書き数字を判別
def predict_digit(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_digit = np.argmax(prediction)
    return predicted_digit

# アップロードされた画像を表示
img = Image.open(image_path)
plt.imshow(img, cmap='gray')
plt.title('Uploaded Image')
plt.axis('off')
plt.show()

# 予測結果を表示
digit = predict_digit(image_path)
print(f'Predicted digit: {digit}')
